In [3]:
from datasets import load_dataset, concatenate_datasets
from toxicity_detection.trainer import Trainer

train_dataset = load_dataset("parquet", data_files="../../data/train_combined.parquet", split="train")
dev_dataset = load_dataset("parquet", data_files="../../data/dev.parquet", split="train")
languages = ["en", "fi", "de"]
n_train_samples = 10
n_dev_samples = 4

def stratified_sample_equal_per_lang(dataset, languages, total_samples, seed=0):
    """Sample equally from each language in `languages`."""
    num_langs = len(languages)
    base_n = total_samples // num_langs
    remainder = total_samples % num_langs
    per_lang_counts = {lang: base_n for lang in languages}
    for lang in languages[:remainder]:
        per_lang_counts[lang] += 1

    per_lang_datasets = []
    for lang in languages:
        lang_subset = dataset.filter(lambda ex, l=lang: ex["lang"] == l)
        lang_subset = lang_subset.shuffle(seed=seed)
        n = per_lang_counts[lang]
        if n > len(lang_subset):
            raise ValueError(f"Requested {n} samples for language '{lang}', "
                            f"but only {len(lang_subset)} available.")
        per_lang_datasets.append(lang_subset.select(range(n)))

    combined = concatenate_datasets(per_lang_datasets).shuffle(seed=seed)
    return combined

train_samples = stratified_sample_equal_per_lang(
    dev_dataset, languages, total_samples=n_train_samples, seed=0
)
# dev_samples = stratified_sample_equal_per_lang(
#     dev_dataset, languages, total_samples=n_dev_samples, seed=1
# )

trainer = Trainer(
    model_name="google/gemma-3-270m",
    train_dataset=train_samples,
    val_dataset=train_samples,
    languages=["en", "fi", "de"],
    batch_size=1,
    learning_rate=1e-4,
    warmup_steps=1,
    accumulation_steps=2,
    eval_steps=1,
    num_epochs=5,
    max_num_checkpoints=1
)
model, tokenizer = trainer.train_model()


Training on device: cpu


Loading weights: 100%|██████████| 236/236 [00:00<00:00, 828.04it/s, Materializing param=norm.weight]                                



Epoch 1/5


Evaluating: 100%|██████████| 10/10 [00:00<00:00, 10.16it/s].93it/s, loss=2.55, tox_loss=0.653, lang_loss=1.89, labeled=1, step=0]


Validation - Macro F1: 0.4889


...

Validation - Macro F1: 0.6000


Epoch 5/5 (λ=1.000): 100%|██████████| 10/10 [00:24<00:00,  2.45s/it, loss=55.7, tox_loss=0.432, lang_loss=55.2, labeled=1, step=25]



Training - Loss: 39.0322, Toxicity Loss: 0.2150, Language Loss: 38.8173, Labeled Samples: 10


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  9.58it/s]



Validation Metrics:
  Loss: 44.0141
  Toxicity Loss: 0.2113
  Language Loss: 43.8028
  Toxicity Macro F1: 0.6000
  Labeled Samples: 10

Training completed!
